# Vererbung

Vererbung erlaubt es, neue Klassen auf Grundlage anderer zu erstellen.

Die neue Klasse wird oft _Subklasse_ oder _Child-Klasse_ genannt, während die alte Klasse, von welcher "geerbt" wird, _Parent_- oder _Base-Klasse_ genannt wird.

Nehmen wir das Beispiel von `User`n:
* Alle User haben einen `username`.
* Alle User haben ein Profil, das man anschauen kann.

Als Klasse könnte `User` so aussehen:

In [ ]:
from dataclasses import dataclass


@dataclass
class User:
    username: str

    def show_profile(self):
        print(f"This is User {self.username}")

`Admin`s sind auch `User`, besitzen alle Eigenschaften von `User`n und können noch mehr:
* Sie können andere User verwalten.

Dass `Admin`s alle Eigenschaften von `User` erben soll, kann erreicht werden, indem die Parent-Klasse `User` in Klammern neben dem neuen Klassennamen hingeschrieben wird:

In [ ]:
class Admin(User):
    def manage_user(self, other: User, new_username):
        other.username = new_username

Nun können wir Objekte von der Klasse `User` und `Admin` erstellen:

In [ ]:
# User (kein Admin):
user123 = User("user123")
pierre: User = User("pièrre")

# Admins:
tux = Admin("tux")
root: User = Admin("root")
chuck_norris: Admin = Admin("chuck_norris")

Auf allen diesen Objekten können wir das Profil anschauen, da dies bei allen `User`n möglich ist. Die `Admin`s besitzen auch diese Methode, weil die Klasse die Methode von `User` geerbt hat:

In [ ]:
user123.show_profile()
pierre.show_profile()

# Admins:
tux.show_profile()
root.show_profile()
chuck_norris.show_profile()

Die `manage_user(...)`-Medhode hingegen existiert nur für `Admin`s:

In [ ]:
tux.manage_user(user123, "user1")
root.manage_user(user123, "user2")
chuck_norris.manage_user(user123, "user3")

Das Konzept der Vererbung folgt dem Prinzip der Wiederverwendung von Code. Das bedeutet, dass du nicht für alle Subklassen die gleichen Methoden selber neu implementieren musst, sondern, dass es reicht, dass du sie in der Base-Klasse implementierst.

## "Is-A"-Beziehung

Beim Definieren der Variable `root` ist dir sicher aufgefallen, dass wir ein `Admin`-Objekt einer `User`-Variable zugewiesen haben:

```python
root: User = Admin("root")
```

Zu Beginn haben wir gesagt, dass die Variable vom Typ `User` sein soll: `root: User`.

Rechts nach dem `=`-Operator folgt dann aber ein Objekt vom Typ `Admin`. Dies ist zulässig, weil ein `Admin` auch ein `User` ist ("ist-ein"-Beziehung), umgekehrt aber nicht zwingend.

Möchten wir herausfinden, ob ein Objekt eine Instanz von einer bestimmten Klasse ist, dann verwenden wir die `isinstance(object, class)`-Funktion:

In [ ]:
# Prüfen, ob Objekt vom Typ `User` ist:
print( isinstance(user123,      User) )
print( isinstance(pierre,       User) )
print( isinstance(tux,          User) )
print( isinstance(root,         User) )
print( isinstance(chuck_norris, User) )

print("\n")

# Prüfen, ob User Admin ist:
print( isinstance(user123,      Admin) )
print( isinstance(pierre,       Admin) )
print( isinstance(tux,          Admin) )
print( isinstance(root,         Admin) )
print( isinstance(chuck_norris, Admin) )

Diese Eigenschaft kann in sich als sehr nützlich herausstellen.

Z.B. wenn wir eine Funktion schreibt, die etwas mit Objekten macht, die entweder vom Typ `User` oder `Admin` sind:

In [ ]:
def get_all_usernames(users: list[User]) -> list[str]:
    usernames = [user.username for user in users]

    return usernames


get_all_usernames([user123, pierre, tux, root, chuck_norris])

Wie du gesehen hast, werden auch Admins in der Liste im Parameter akzeptiert.

Folglich reicht es vollkommen aus, diese Methode 1-mal zu implementieren. Zusätzlich wird sie für alle weiteren Sub-Klassen funktionieren, die in der Zukunft geschrieben wird.

Mit Vererbung können wir in vielen Fällen sehr viel Zeilen Code sparen.

## Methoden überschreiben
Oft möchtest du eine Methode in einer Sub-Klasse ein bisschen abändern. Das kannst du machen, indem du die genau gleiche Methode in der Sub-Klasse neu definierst (hier `get_privileges()`):

In [ ]:
from dataclasses import dataclass
from typing_extensions import override


@dataclass
class User:
    username: str

    def get_privileges(self):
        return ["read"]


class Admin(User):
    @override
    def get_privileges(self):
        return ["read", "write"]


Wenn du nun `get_privileges()` auf einem `User`-Objekt aufrufst, dann wird die Methode in `User` aufgerufen.

Da du diese Methode in `Admin` überschrieben hast, wird die neue Methode aufgerufen, wenn du die Methode im `Admin`-Objekt aufrufst:

In [ ]:
user123 = User("user123")
root: User = Admin("root")

print( user123.get_privileges() )
print( root.get_privileges()    )


Beachte, dass die `@override`-Annotation freiwillig ist. Es ist aber eine Good-Practice, diese Angabe zu machen, damit andere Entwickler:innen sehen, dass die Methode überschrieben wird.

Möchtest du, dass in deiner neuen Methode auch die überschriebene Methode aufgerufen wird, dann kannst du in der neuen Methode den Aufruf mit `super.Methoden-Name(Argumente)` ergänzen. In unserem Beispiel müsste das `super().get_privileges()` sein.

## protected

In der Theorie zu Variablen in Klassen wurden verschiedene Sichtbarkeitstufen vorgestellt (public, protected und private).

Nun stellen wir dir noch die _protected_-Sichtbarkeitsstufe vor. Methodennamen mit der Sichtbarkeitsstufe _protected_ beginnen mit 1 Underscore (`_`):

In [ ]:
from dataclasses import dataclass


@dataclass
class User:
    _username: str

    def get_username(self):
        return self._username


class Admin(User):
    def username_in_capital_letters(self):
        return self._username.upper()


_Protected_ bedeutet in diesem Fall, dass auch die Sub-Klassen (hier `Admin`) die Variable `_username` sehen kann:

In [ ]:
root = Admin("root")

print (root.get_username()                  )
print (root.username_in_capital_letters()   )

Die Meinung hinter der Sichtbarkeitsstufe _protected_ ist, dass nur die Parent- und Sub-Klasse die Variable `_username` sehen können sollen.

Folglich sollte folgender Code vermieden werden, obwohl er funktioniert:

In [ ]:
root._username

Wichtig zu verstehen ist, dass die Sichtbarkeitsstufe _protected_ nicht wirklich eine Python-Funktion ist, sondern, dass sich die Entwickler:innen daran halten sollen, keine _protected_ Felder aufzurufen.

Wenn du also in deinem Code eine Methode oder eine Variable von einem Objekt verwendest, die mit einem Underscore beginnt, dann verletzt du dieses Prinzip. Ausser: du befindest dich in der gleichen Klasse, in welcher die Variable definiert wird, oder in einer Sub-Klasse von ihr.